### Одноклассовый классификатор на основе трехслойного автоэнкодера

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
np.bool = np.bool_
np.int = np.int_
np.object = np.object_
import tensorflow as tf

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model
from tensorboard.plugins.hparams import api as hp

#### Загрузка данных

In [ ]:
normal_train_data = tf.convert_to_tensor(np.load('../data/normal_train_data.npy'))
normal_test_data = tf.convert_to_tensor(np.load('../data/normal_test_data.npy'))
anomalous_train_data = tf.convert_to_tensor(np.load('../data/anomalous_train_data.npy'))
anomalous_test_data = tf.convert_to_tensor(np.load('../data/anomalous_test_data.npy'))
train_data = np.load('../data/train_data.npy')
test_data = tf.convert_to_tensor(np.load('../data/test_data.npy'))
train_labels = np.load('../data/train_labels.npy')
test_labels = tf.convert_to_tensor(np.load('../data/test_labels.npy'))
from sklearn.utils import shuffle

In [ ]:

from sklearn.decomposition import PCA

t_d_1 = tf.convert_to_tensor(np.concatenate((normal_train_data[:500].numpy(),anomalous_train_data[:500].numpy())))
y = tf.convert_to_tensor(np.concatenate((np.ones(500, dtype=int), np.zeros(500, dtype=int))))

num_bits = 7

pca = PCA(n_components=num_bits)  # Set number of components to 24
train_data_reduced = tf.convert_to_tensor(pca.fit_transform(t_d_1, y))
validation_data_reduced = tf.convert_to_tensor(pca.fit_transform(test_data, test_labels))

In [9]:



class BinaryClassfier(Model):
  def __init__(self, x):
    super(BinaryClassfier, self).__init__()

    self.encoder = tf.keras.Sequential([layers.Dense(x // 2, activation="relu"),
      layers.Dense(1, activation="relu")])

  def call(self, x):
    encoded = self.encoder(x)
    return encoded

  
def model_builder(num_bits):
  
  model = BinaryClassfier(num_bits)
  model.compile(optimizer='adam', loss='mean_squared_error', metrics= ["accuracy"])
  return model

autoencoder = model_builder(num_bits)
history = autoencoder.fit(train_data_reduced, y, epochs=30,
          validation_data=(validation_data_reduced, test_labels))

Epoch 1/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.5058 - loss: 0.4702 - val_accuracy: 0.4200 - val_loss: 0.5558
Epoch 2/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5167 - loss: 0.4318 - val_accuracy: 0.4220 - val_loss: 0.4853
Epoch 3/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5226 - loss: 0.3769 - val_accuracy: 0.4380 - val_loss: 0.3650
Epoch 4/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5267 - loss: 0.3015 - val_accuracy: 0.4810 - val_loss: 0.2812
Epoch 5/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6149 - loss: 0.2316 - val_accuracy: 0.6000 - val_loss: 0.2310
Epoch 6/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7242 - loss: 0.1878 - val_accuracy: 0.7090 - val_loss: 0.1973
Epoch 7/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7695 - loss: 0.1757 - val_accuracy: 0.7850 - val_loss: 0.1729
Epoch 8/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8299 - loss: 0.1510 - val_accuracy: 0.8220 - val_loss: